In [1]:
import numpy as np
import pandas as pd
from ggplot import *

In [2]:
dataComp = pd.read_csv("output/competitions.csv")
dataSubm = pd.read_csv("output/submissions.csv")
dataEval = pd.read_csv("output/EvaluationAlgorithms.csv")
dataTeams = pd.read_csv("output/Teams.csv")
dataUsers = pd.read_csv("output/Users.csv")

/Library/Python/2.7/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (4,5,9) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


In [3]:
dataComp.DateEnabled = dataComp.DateEnabled.apply(lambda x: pd.to_datetime(x[0:10]))
dataComp.Deadline = dataComp.Deadline.apply(lambda x: pd.to_datetime(x[0:10]))

In [4]:
EvalAlgs = (dataComp.groupby('EvaluationAlgorithmId').apply(lambda x: len(x)))
EvalAlgs.sort(ascending = 0)
print (EvalAlgs.head())
print (EvalAlgs.sum())

EvaluationAlgorithmId
5     46
2     36
14    36
6     16
25    11
dtype: int64
239


In [5]:
dataEval[dataEval.Id == 5]

,Id,Abbreviation,Name,Description,IsMax
3,5,AUC,Area Under Receiver Operating Characteristic C...,Measures discrimination. Calculates how well a...,True


In [6]:
dataEval[dataEval.Id == 2]

,Id,Abbreviation,Name,Description,IsMax
1,2,RMSE,Root Mean Squared Error,Square root of the average of the squared erro...,False


In [7]:
dataEval[dataEval.Id == 14]

,Id,Abbreviation,Name,Description,IsMax
8,14,CategorizationAccuracy,Categorization Accuracy,Percentage of correctly categorized items,True


In [40]:
dataEval[dataEval.Id == 6]

,Id,Abbreviation,Name,Description,IsMax
4,6,MAE,Mean Absolute Error,Average of the absolute value of each individu...,False


In [58]:
def findMaxScores(IDList, teamNumber):
    beta = pd.DataFrame(data = [], columns = dataSubm.columns.values.tolist() + ['CompetitionId'])
    for compID in IDList:
        curCompTeams = dataTeams[dataTeams.CompetitionId == compID][dataTeams.Ranking <= teamNumber]
        for teamID in curCompTeams.Id:
            K = dataSubm[dataSubm.Id == dataSubm.Id[dataSubm[dataSubm.TeamId == teamID].PrivateScore.idxmax()]]
            K['CompetitionId'] = int(compID)
            beta = beta.append(K)
    
    return beta

def findMinScores(IDList, teamNumber):
    beta = pd.DataFrame(data = [], columns = dataSubm.columns.values.tolist() + ['CompetitionId'])
    for compID in IDList:
        curCompTeams = dataTeams[dataTeams.CompetitionId == compID][dataTeams.Ranking <= teamNumber]
        for teamID in curCompTeams.Id:
            K = dataSubm[dataSubm.Id == dataSubm.Id[dataSubm[dataSubm.TeamId == teamID].PrivateScore.idxmin()]]
            K['CompetitionId'] = int(compID)
            beta = beta.append(K)
    
    return beta

In [9]:
AUCComps = dataComp[dataComp.EvaluationAlgorithmId == 5]

In [10]:
AUCsc = findMaxScores (AUCComps.Id, 5)

/Library/Python/2.7/site-packages/pandas/core/frame.py:1825: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  "DataFrame index.", UserWarning)
/Library/Python/2.7/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [11]:
AUCsc.PrivateScore = AUCsc.PrivateScore.apply(lambda x: float(x))
AUCsc.PublicScore = AUCsc.PublicScore.apply(lambda x: float(x))

In [12]:
AUCDF = AUCsc[['PrivateScore', 'PublicScore', 'CompetitionId']].copy().groupby('CompetitionId').mean()

In [43]:
dataComp[dataComp.Id == 2752]

,Id,CompetitionName,Title,BriefDescription,DateEnabled,Deadline,HasLeaderboard,SolutionNumRows,EvaluationAlgorithmId,LeaderboardPercentage,...,RewardQuantity,CompetitionHostSegmentId,SolutionLastColumnWasLeaderboardUsageIndicator,ModelSubmissionDeadline,EnableSubmissionModelHashes,EnableSubmissionModelAttachments,MaxTeamSize,HyperParameters,CanQualifyTalent,EvaluationAlgorithmParameters
41,2752,webspam,Обнаружение веб-спама,"Соревнование в рамках курса ШАД ""Веб-Графы""",2012-02-20,2012-06-01,True,114529,5,2,...,NaN,1,False,NaN,False,False,NaN,NaN,False,NaN


In [15]:
dataComp[dataComp.Id == 3126]

,Id,CompetitionName,Title,BriefDescription,DateEnabled,Deadline,HasLeaderboard,SolutionNumRows,EvaluationAlgorithmId,LeaderboardPercentage,...,RewardQuantity,CompetitionHostSegmentId,SolutionLastColumnWasLeaderboardUsageIndicator,ModelSubmissionDeadline,EnableSubmissionModelHashes,EnableSubmissionModelAttachments,MaxTeamSize,HyperParameters,CanQualifyTalent,EvaluationAlgorithmParameters
72,3126,algorithmic-composition,Algorithmic composition,Классификация с помощью алгоритмических композ...,2012-09-11,2012-11-09,True,551,5,100,...,1,1,True,NaN,False,False,NaN,<parameters/>,False,NaN


In [16]:
dataComp[dataComp.Id == 3507]

,Id,CompetitionName,Title,BriefDescription,DateEnabled,Deadline,HasLeaderboard,SolutionNumRows,EvaluationAlgorithmId,LeaderboardPercentage,...,RewardQuantity,CompetitionHostSegmentId,SolutionLastColumnWasLeaderboardUsageIndicator,ModelSubmissionDeadline,EnableSubmissionModelHashes,EnableSubmissionModelAttachments,MaxTeamSize,HyperParameters,CanQualifyTalent,EvaluationAlgorithmParameters
120,3507,the-icml-2013-bird-challenge,The ICML 2013 Bird Challenge,Identify bird species from continuous audio re...,2013-05-08,2013-06-17,True,3150,5,33,...,500,2,True,NaN,False,False,NaN,<parameters/>,True,NaN


In [17]:
dataComp[dataComp.Id == 3933]

,Id,CompetitionName,Title,BriefDescription,DateEnabled,Deadline,HasLeaderboard,SolutionNumRows,EvaluationAlgorithmId,LeaderboardPercentage,...,RewardQuantity,CompetitionHostSegmentId,SolutionLastColumnWasLeaderboardUsageIndicator,ModelSubmissionDeadline,EnableSubmissionModelHashes,EnableSubmissionModelAttachments,MaxTeamSize,HyperParameters,CanQualifyTalent,EvaluationAlgorithmParameters
171,3933,mlsp-2014-mri,MLSP 2014 Schizophrenia Classification Challenge,Diagnose schizophrenia using multimodal featur...,2014-06-05,2014-07-20,True,2,5,0,...,0,2,True,NaN,False,False,NaN,NaN,True,"<parametersSchema type=""Kaggle.Metrics.EmptyEv..."


In [18]:
AUCDF['Label'] = ''

In [19]:
AUCDF.Label[2752] = 'Web-spam diagnostic'
AUCDF.Label[3126] = 'Algorithmic composition'
AUCDF.Label[3507] = 'The ICML 2013 \n Bird Challenge'
AUCDF.Label[3933] = 'MLSP 2014 Schizophrenia Classification'

/Library/Python/2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Library/Python/2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Library/Python/2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/Library/Python/2.7/site-packages/ipykernel/__main__.py:4: SettingWithC

In [20]:
AUCplot = ggplot(aes(x='PrivateScore',y = 'PublicScore', label = 'Label'), data=AUCDF)
AUCplot + geom_point() + xlim(-0.1,1.1) + ylim(-0.1,1.1) + geom_text(hjust=-0.1, vjust=-0.05)

<ggplot: (285333665)>

In [21]:
CAComps = dataComp[dataComp.EvaluationAlgorithmId == 14]

In [22]:
CAsc = findMaxScores (CAComps.Id, 5)

/Library/Python/2.7/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [23]:
CAsc.PrivateScore = CAsc.PrivateScore.apply(lambda x: float(x))
CAsc.PublicScore = CAsc.PublicScore.apply(lambda x: float(x))

In [24]:
CADF = CAsc[['PrivateScore', 'PublicScore', 'CompetitionId']].copy().groupby('CompetitionId').mean()

In [25]:
dataComp[dataComp.Id == 3439]

,Id,CompetitionName,Title,BriefDescription,DateEnabled,Deadline,HasLeaderboard,SolutionNumRows,EvaluationAlgorithmId,LeaderboardPercentage,...,RewardQuantity,CompetitionHostSegmentId,SolutionLastColumnWasLeaderboardUsageIndicator,ModelSubmissionDeadline,EnableSubmissionModelHashes,EnableSubmissionModelAttachments,MaxTeamSize,HyperParameters,CanQualifyTalent,EvaluationAlgorithmParameters
109,3439,handwritten-digits-recognition,Handwritten digits recognition,В этом задании вам предстоит реализовать алгор...,2013-03-15,2013-05-13,True,3177,14,100,...,NaN,1,True,NaN,False,False,NaN,<parameters/>,False,NaN


In [26]:
dataComp[dataComp.Id == 3497]

,Id,CompetitionName,Title,BriefDescription,DateEnabled,Deadline,HasLeaderboard,SolutionNumRows,EvaluationAlgorithmId,LeaderboardPercentage,...,RewardQuantity,CompetitionHostSegmentId,SolutionLastColumnWasLeaderboardUsageIndicator,ModelSubmissionDeadline,EnableSubmissionModelHashes,EnableSubmissionModelAttachments,MaxTeamSize,HyperParameters,CanQualifyTalent,EvaluationAlgorithmParameters
118,3497,missing-and-imbalanced-data,Missing and Imbalanced Data,Вам предстоит решить задачу классификации для ...,2013-04-26,2013-05-13,True,8000,14,100,...,NaN,1,True,NaN,False,False,NaN,<parameters/>,False,NaN


In [27]:
CADF['Label'] = ''

In [28]:
CADF.Label[3439] = 'Handwritten digits recognition'
CADF.Label[3497] = 'Missing and Imbalanced Data'

/Library/Python/2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Library/Python/2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [29]:
CAplot = ggplot(aes(x='PrivateScore',y = 'PublicScore', label = 'Label'), data=CADF)
CAplot + geom_point() + xlim(-0.1,1.1) + ylim(-0.1,1.1) + geom_text(vjust=-0.05)

<ggplot: (283489985)>

In [30]:
RMSEComps = dataComp[dataComp.EvaluationAlgorithmId == 2]

In [59]:
RMSEsc = findMinScores (RMSEComps.Id, 5)

/Library/Python/2.7/site-packages/ipykernel/__main__.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [60]:
RMSEsc.PrivateScore = RMSEsc.PrivateScore.apply(lambda x: float(x))
RMSEsc.PublicScore = RMSEsc.PublicScore.apply(lambda x: float(x))

In [61]:
RMSEDF = RMSEsc[['PrivateScore', 'PublicScore', 'CompetitionId']].copy().groupby('CompetitionId').mean()

In [62]:
RMSEDF

,PrivateScore,PublicScore
CompetitionId,,
2447,0.695284,0.659395
2467,192.347198,207.376001
2487,0.015141,0.015042
2554,0.086726,0.086693
2555,3.051102,2.821722
2559,1.159176,1.240734
2589,0.772638,0.766263
2730,4.473191,4.633297
2963,13.276346,13.243869


In [64]:
dataComp[dataComp.Id == 4272]

,Id,CompetitionName,Title,BriefDescription,DateEnabled,Deadline,HasLeaderboard,SolutionNumRows,EvaluationAlgorithmId,LeaderboardPercentage,...,RewardQuantity,CompetitionHostSegmentId,SolutionLastColumnWasLeaderboardUsageIndicator,ModelSubmissionDeadline,EnableSubmissionModelHashes,EnableSubmissionModelAttachments,MaxTeamSize,HyperParameters,CanQualifyTalent,EvaluationAlgorithmParameters
211,4272,restaurant-revenue-prediction,Restaurant Revenue Prediction,Predict annual restaurant sales based on objec...,2015-03-23,2015-05-04,True,2,2,0,...,30000,1,True,NaN,False,False,NaN,NaN,True,"<parametersSchema type=""Kaggle.Metrics.EmptyEv..."


In [57]:
dataSubm[dataSubm.TeamId == 86300]

,Id,SubmittedUserId,DateSubmitted,TeamId,PrivateScore,PublicScore,IsSelected,ScoreStatus,IsAfterDeadline,DateScored,ScoringDurationMilliseconds
342094,671442,182876,2014-04-08 11:09:39.266666,86300,0.007525926532357259,0.009601787620943153,False,1,False,2014-04-08 11:09:40.6515119,181


In [34]:
RMSEplot = ggplot(aes(x='PrivateScore',y = 'PublicScore'), data=RMSEDF)
RMSEplot + geom_point() 

<ggplot: (283480513)>